# Set connection with GitHub

In [41]:
# ! cd /kaggle/working && rm -rf RECsys_Challenge2024

In [42]:
from kaggle_secrets import UserSecretsClient

token = UserSecretsClient().get_secret("Token")

! git clone https://{token}@github.com/madratak/RECsys_Challenge2024.git

Cloning into 'RECsys_Challenge2024'...
remote: Enumerating objects: 1706, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 1706 (delta 50), reused 36 (delta 16), pack-reused 1618 (from 1)
Receiving objects: 100% (1706/1706), 67.98 MiB | 21.69 MiB/s, done.
Resolving deltas: 100% (995/995), done.


In [43]:
!pip install PyGithub requests

In [44]:
!cd RECsys_Challenge2024 && python run_compile_all_cython.py

run_compile_all_cython: Found 11 Cython files in 5 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/11]: MatrixFactorizationImpressions_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1929,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorizationImpressions_Cython_Epoch.c:1252:
/opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorizationImpressions_Cython_Epoch.c: In functio

In [45]:
import numpy as np
import pandas as pd
import scipy.sparse as sps
import time
import shutil

np.random.seed(42)

## Import the repository

In [46]:
from github import Github, Auth

# Authenticate using a personal access token
auth_token = Auth.Token(token)
github_client = Github(auth=auth_token)

# Define the repository name you want to find
target_repo_name = 'RECsys_Challenge2024'
repo = None

# Search for the repository in the user's repositories
try:
    for repository in github_client.get_user().get_repos():
        if repository.name == target_repo_name:
            repo = repository
            print(f"Repository '{target_repo_name}' found.")
            break
    if repo is None:
        print(f"Repository '{target_repo_name}' not found.")
except Exception as e:
    print("An error occurred while accessing the repositories:", e)

Repository 'RECsys_Challenge2024' found.


In [47]:
def upload_file(filepath_kaggle, filepath_github, commit_message):
    """
    Uploads a file from Kaggle to GitHub, us if it already exists in the repository,
    or creating it if it does not.

    Parameters:
    - filepath_kaggle: Path to the file in the Kaggle environment.
    - filepath_github: Target path in the GitHub repository where the file should be uploaded.
    - commit_message: Message for the commit on GitHub.
    """
    try:
        
        # Check if the file already exists in the GitHub repository
        contents = repo.get_contents(filepath_github)
        
        # If it exists, update the file
        with open(filepath_kaggle, "rb") as file:
            repo.update_file(
                contents.path, commit_message, file.read(), contents.sha
            )
        print(f"File '{filepath_github}' updated successfully.")
    
    except Exception as e:
        
        # If the file does not exist, create it
        with open(filepath_kaggle, "rb") as file:
            repo.create_file(
                filepath_github, commit_message, file.read()
            )
        print(f"File '{filepath_github}' created successfully.")

In [48]:
config = {
    'model': 'P3alphaRecommender',
    'tune_parameters': True,
    'database_path': '/kaggle/working/history_P3alphaRecommender.db',
    'copy_prev_best_params': False,
    'save_github': True
}

Import the database where previous tuning trials have been saved.

In [49]:
try:
    shutil.copyfile(
        '/kaggle/working/Recsys_Challenge_2023/TrainedModels/' \
        '{config["model"]}Recommender/history_{config["model"]}.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

# Construction of URM and ICM matrices

In [50]:
URM_all_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_train.csv")

n_users = len(URM_all_dataframe["user_id"].unique())
n_items = len(URM_all_dataframe["item_id"].unique())

URM_all = sps.csr_matrix((URM_all_dataframe["data"].values, 
                          (URM_all_dataframe["user_id"].values, URM_all_dataframe["item_id"].values)),
                        shape = (n_users, n_items))

URM_all

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1764607 stored elements and shape (35736, 38121)>

In [51]:
ICM_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_ICM_metadata.csv")

n_items = len(ICM_dataframe["item_id"].unique())
n_features = len(ICM_dataframe["feature_id"].unique())

ICM_all = sps.csr_matrix((ICM_dataframe["data"].values, 
                          (ICM_dataframe["item_id"].values, ICM_dataframe["feature_id"].values)),
                        shape = (n_items, n_features))

ICM_all

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2940040 stored elements and shape (38121, 94331)>

# Training

In [52]:
cd /kaggle/working/RECsys_Challenge2024/

/kaggle/working/RECsys_Challenge2024


In [53]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions


In [54]:
import optuna
import pandas as pd
# from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender

def objective_function_P3alphaRecommender(optuna_trial):
    
    recommender_instance = P3alphaRecommender(URM_train)
    
    full_hyperp = {
                   # "topK": 10,  # Fixed value, not tuned
                   "topK": optuna_trial.suggest_int("topK", 0, 1500),
                   "alpha": optuna_trial.suggest_float("alpha", 0, 2, log=False),
                   "normalize_similarity": optuna_trial.suggest_categorical("normalize_similarity", [True, False]),
                   "implicit": optuna_trial.suggest_categorical("implicit", [True, False]),

    }

    recommender_instance.fit(**full_hyperp)
    
    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)
    
    return result_df.loc[10]["MAP"]

In [58]:
if config['tune_parameters']:
    
    study = optuna.create_study(direction='maximize', study_name=f'hyperparameters_tuning_{config["model"]}',
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)

    study.optimize(objective_function_P3alphaRecommender, n_trials=30)

[I 2024-11-15 11:01:48,206] Using an existing study with name 'hyperparameters_tuning_P3alphaRecommender' instead of creating a new one.


P3alphaRecommender: Similarity column 38121 (100.0%), 2278.43 column/sec. Elapsed time 16.73 sec
EvaluatorHoldout: Processed 35595 (100.0%) in 34.20 sec. Users per second: 1041


[I 2024-11-15 11:02:40,693] Trial 1 finished with value: 0.042880060513223746 and parameters: {'alpha': 0.39144034003803685, 'normalize_similarity': False, 'implicit': True}. Best is trial 0 with value: 0.04752105699703477.


P3alphaRecommender: Similarity column 38121 (100.0%), 2270.89 column/sec. Elapsed time 16.79 sec
EvaluatorHoldout: Processed 35595 (100.0%) in 34.33 sec. Users per second: 1037


[I 2024-11-15 11:03:33,389] Trial 2 finished with value: 0.05289676073641322 and parameters: {'alpha': 0.5890939043550703, 'normalize_similarity': True, 'implicit': False}. Best is trial 2 with value: 0.05289676073641322.


...

[I 2024-11-15 11:26:14,750] Trial 28 finished with value: 0.05303740381763789 and parameters: {'alpha': 0.5463379036070167, 'normalize_similarity': True, 'implicit': True}. Best is trial 22 with value: 0.05334599451055225.


P3alphaRecommender: Similarity column 38121 (100.0%), 2282.06 column/sec. Elapsed time 16.70 sec
EvaluatorHoldout: Processed 35595 (100.0%) in 33.85 sec. Users per second: 1052


[I 2024-11-15 11:27:06,861] Trial 29 finished with value: 0.05324293027600889 and parameters: {'alpha': 0.16962864137322087, 'normalize_similarity': True, 'implicit': False}. Best is trial 22 with value: 0.05334599451055225.


P3alphaRecommender: Similarity column 38121 (100.0%), 2284.28 column/sec. Elapsed time 16.69 sec
EvaluatorHoldout: Processed 35595 (100.0%) in 33.99 sec. Users per second: 1047


[I 2024-11-15 11:27:59,127] Trial 30 finished with value: 0.05252759104297052 and parameters: {'alpha': 0.6893996849354879, 'normalize_similarity': True, 'implicit': False}. Best is trial 22 with value: 0.05334599451055225.


In [60]:
print(study.best_trial.params)

{'alpha': 0.369533359594755, 'normalize_similarity': True, 'implicit': True}


In [64]:
recommender_instance = P3alphaRecommender(URM_train + URM_validation)
recommender_instance.fit(**study.best_trial.params)

P3alphaRecommender: Similarity column 38121 (100.0%), 1441.73 column/sec. Elapsed time 26.44 sec


# Testing

Create the recommendations for the submission. 

In [66]:
def create_submission(data_target_users_test, recommender_instance, cutoff=10, output_file=f"/kaggle/working/submission_{config['model']}_fixed_topk.csv"):
    
    target_result = []

    for target in data_target_users_test["user_id"]:
        target_result.append(recommender_instance.recommend(target, cutoff=cutoff, remove_seen_flag=True))

    user_ids = data_target_users_test["user_id"]
    formatted_data = {
        "user_id": user_ids,
        "item_list": [" ".join(map(str, items)) for items in target_result]
    }

    submission_df = pd.DataFrame(formatted_data)
    submission_df.to_csv(output_file, index=False, header=["user_id", "item_list"])

    print(f"Submission file saved as {output_file}")

In [70]:
data_target_users_test = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_target_users_test.csv")

create_submission(data_target_users_test, recommender_instance)

# upload_file(
#             f'/kaggle/working/submission_{config["model"]}_fixed_topk.csv', 
#             f'/kaggle/working/Recsys_Challenge_2023/TrainedModels/{config["model"]}Recommender/'\
#                 f'submission_{config["model"]}.csv', 
#             f'New {config["model"]} submission (from kaggle notebook)'
#         )

Submission file saved as /kaggle/working/submission_P3alphaRecommender_fixed_topk.csv


# Save Version on GitHub 

Write or import a json file where best hyperparameters are saved. 

In [72]:
import json

if config['tune_parameters']:
    with open(f'/kaggle/working/best_params_{config["model"]}.json', 'w') as params_file:
        json.dump(study.best_params, params_file)
        
    if config['save_github']:
        upload_file(
            f'/kaggle/working/best_params_{config["model"]}.json', 
            f'TrainedModels/{config["model"]}Recommender/best_params_{config["model"]}.json', 
            f'{config["model"]} tuning results (from kaggle notebook)'
        )
elif config['copy_prev_best_params']:
    shutil.copyfile(
        f'/kaggle/working/RECsys_Challenge2024/TrainedModels/{config["model"]}Recommender/'\
        f'best_params_{config["model"]}.json', 
        f'/kaggle/working/best_params_{config["model"]}.json'
    )

File 'TrainedModels/P3alphaRecommenderRecommender/best_params_P3alphaRecommender.json' created successfully.


Save the history of the tuned model.

In [73]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'], 
        f'tuning_results/history_{config["model"]}.db',
        'Tuning db updated results (from kaggle notebook)'
    )

File 'tuning_results/history_P3alphaRecommender.db' created successfully.


In [76]:
if config['save_github']:
    upload_file(
                f'/kaggle/working/submission_{config["model"]}_fixed_topk.csv', 
                f'TrainedModels/{config["model"]}Recommender/Submission/'\
                    f'submission_{config["model"]}_fixed_topk.csv', 
                f'New {config["model"]} submission (from kaggle notebook)'
            )

File 'TrainedModels/P3alphaRecommenderRecommender/Submission/submission_P3alphaRecommender_fixed_topk.csv' created successfully.
